In [5]:
import itertools
import pandas as pd

from sklearn import (
    model_selection, linear_model, metrics
)

## loading data

In [6]:
df1 = 'AimoScore_WeakLink_big_scores (1).xls'
df1 = pd.read_excel(df1)

df2 = '20190108 scores_and_weak_links.xlsx'
df2 = pd.read_excel(df2)

c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## applying the train test split

In [7]:
X1 = df1.drop(['AimoScore','EstimatedScore'], axis= 1)
Y1 = df1['AimoScore']

X2 = df2.drop(['ID','Date','SCORE'], axis= 1)
Y2 = X2.values.argmax(axis= 1)

trainX1, testX1, trainY1, testY1 = model_selection.train_test_split(X1, Y1, test_size= 0.2)
trainX2, testX2, trainY2, testY2 = model_selection.train_test_split(X2, Y2, test_size= 0.2)

# Improving models using cross validation

## champion regression model

In [8]:
losses    = ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
penalties = ['l2', 'l1', 'elasticnet', None]

loss, penalty = max(
    itertools.product(losses,penalties), 
    key= lambda e: model_selection.cross_val_score(
        linear_model.SGDRegressor(loss= e[0], penalty= e[1]), 
        trainX1, 
        trainY1
    ).mean()
)

print(f'the maximum score is achieved with loss {loss} and penalty {penalty}')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 1585, in fit
    return self._fit(
  File "c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 1530, in _fit
    self._partial_fit(
  File "c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 1435, in _partial_fit
    X, y = self._validate_da

In [ ]:
model = linear_model.SGDRegressor(loss= 'epsilon_insensitive', penalty= 'l2').fit(trainX1,trainY1)
model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

## champion classsification model

In [ ]:
penalties = ['l2', 'l1', None]

penalty = max(
    penalties, 
    key= lambda e: model_selection.cross_val_score(
        linear_model.LogisticRegression(penalty= e, solver= 'saga'), 
        trainX2, 
        trainY2
    ).mean()
)

print(f'the maximum score is achieved with penalty {penalty}')

In [ ]:
model = linear_model.LogisticRegression(penalty= 'l1', solver= 'saga')
model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)